#Loading the required libraries

In [ ]:
import gc
gc.collect()

46

In [ ]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!pip install pattern
!pip install asteval

     |████████████████████████████████| 911kB 18.8MB/s 
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:10 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [52.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://archive.u

In [ ]:
import selenium
import time
import pandas as pd
from tqdm import tqdm
import numpy as np

In [ ]:
# Creating Driver Instance
# install chromium, its driver, and selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium
# set options to be headless, ..
from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
# open it, go to a website, and get results

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

# Loading the movies dataset

In [ ]:
def load_data():
  import pandas as pd
  global movie
  global link
  movie = pd.read_csv('/content/drive/MyDrive/MovieLens Dataset(1)/movie.csv')
  link = pd.read_csv('/content/drive/MyDrive/MovieLens Dataset(1)/link.csv')
  global movie_links
  movie_links = pd.concat([movie,link],axis=1,join='inner')
  movie_links.drop('tmdbId',axis=1,inplace=True)
  df = movie_links['title'].str.split('(',expand=True)
  df.rename({0:'title',1:'year'},axis=1,inplace=True)
  df = df[['title','year']]
  df['year'] = df['year'].str.replace(')','')
  movie_links['year'] = df['year']
  movie_links['title'] = df['title']
  movie_links.index = movie_links['title']




#Scraping the Data

In [ ]:
def extend_page(wd):
  count = 20
  from selenium.common.exceptions import NoSuchElementException,ElementNotInteractableException
  # rev1 = []
  # rev1 = (wd.find_elements_by_xpath("//div[@class='text show-more__control clickable']"))
  # headings = []
  # headers = wd.find_elements_by_class_name('title')
  # headings.extend(headers)
  prev_height = wd.execute_script('return document.body.scrollHeight')


  while(count>0):
    # wd.find_element_by_id("load-more-trigger").click()
    # print(count)
    try:
      # print('In the try block')
      # button = wd.find_element_by_class_name('ipl-load-more__button')
      # button = wd.find_element_by_class_name("ipl-load-more__button")

      # print('button length:',len(button))

      wd.find_element_by_id("load-more-trigger").click()
      print(count)
      # driver.find_element_by_id("load-more-trigger").click()
      # button.click()

    except (ElementNotInteractableException,NoSuchElementException):
      # print('Click event not instantiated')
      page = wd.page_source
      return page
    # print(page[:10])
      # reviews = scrape_reviews(page)
      # if(len(reviews)>=200):
      #   return page

      # else:
      #   return 0
    time.sleep(1)

    wd.execute_script('window.scrollTo(0,document.body.scrollHeight);')
    # time.sleep(1)
    new_height = wd.execute_script('return document.body.scrollHeight')

    # if new_height == prev_height:
    #   print('Height limit reached')
    #   break
    # headers = wd.find_elements_by_class_name('title')
    # print(headers[-1].text)
    # headings.extend(headers)
    # print(headings[-1].text)
    prev_height = new_height
    count-=1
  page = wd.page_source
  return page


In [ ]:
def scrape_reviews(page):
  from bs4 import BeautifulSoup as bsoup
  import requests
  soup = bsoup(page,'lxml')
  revs = soup.find_all('div',{'class':'text show-more__control'})
  revs2 = soup.find_all('div',{'class':'text show-more__control clickable'})

  reviews = []
  global k
  k=0
  for j,i in tqdm(enumerate(revs)):
    reviews.append(i.text)
    k+=1

  for j,i in tqdm(enumerate(revs2)):
    reviews.append(i.text)
    k+=1

  print(k,'reviews scraped')

  return reviews

# Preprocessing

In [ ]:
def preprocess(reviews):
    import re
    import gensim
    from gensim.parsing.preprocessing import remove_stopwords
    from gensim.models.phrases import Phrases,Phraser
    from gensim.utils import lemmatize
    from nltk.stem import WordNetLemmatizer
    lemmatizer = WordNetLemmatizer()
    cleaned_reviews = []
    for rev in reviews:
        text = re.sub('[^a-zA-Z]',' ',rev)
        text = text.lower()
        text = re.sub('(\\d|\\W)+',' ',text)
        text = remove_stopwords(text)
        cleaned_reviews.append(text.split())

    print('text has been cleaned')


#     print('cleaned_reviews:',cleaned_reviews)



    lemmatized_reviews = []


#     lem_without_tags_reviews = []


    for review in cleaned_reviews:
        lemmatized_review = []
#         print('review:',review)
        for word in review:
#             print(word)
            lemmatized_review.append(lemmatizer.lemmatize(word))
#             print(lemmatized_review)
    #         lemmatized_reviews.append(lemmatize(review,allowed_tags=re.compile('.*')))
        lemmatized_reviews.append(lemmatized_review)
        
#     print(len(lemmatized_reviews))
#     print(lemmatized_reviews)

    print('text has been lemmatized')






# #     lemmatized_words = []
# #     for review in lemmatized_reviews:
# #         line = []
# #         for word in review:
# # #     #           line.append(word.decode('utf-8'))
# #           line.append(word)


# #         lemmatized_words.append(line)
        
        


#     for lines in lemmatized_reviews:
#         line = []

#         for word in lines:
# #             print('word:',word)
#             line.append(word.split('/')[0])
            
# #             print('line:',line)


#         lem_without_tags_reviews.append(line)



    phrases = Phrases(lemmatized_reviews)
    print('bigram step 1')
    bigram = Phraser(phrases)
    print('bigram step 2')


    final_phrases = bigram[lemmatized_reviews]

    print('bigrams have been created')

    print('type:',type(final_phrases))

    
    return final_phrases


# Data Labelling

In [ ]:
def labelling(reviews):
  print('labelling')
  import nltk
  import pandas as pd
  # nltk.download('all')
  nltk.download('vader_lexicon')

  from nltk.sentiment.vader import SentimentIntensityAnalyzer

  sid = SentimentIntensityAnalyzer()

  labels = []

  for review in reviews:

    score = sid.polarity_scores(review)

    if (score['compound'] >= 0.05 ):
      labels.append(1)

    elif (score['compound'] <= -0.05):
      labels.append(2)

    else:
      labels.append(0)


  df = pd.DataFrame()
  print('reviews:',len(reviews))
  print('labels:',len(labels))


  df['reviews'] = reviews
  df['label'] = labels

  return df





# embedding

In [ ]:
def embedding(all_phrases1):
  print('embedding')
  print('Shape of phrases',all_phrases1.shape)
  import gensim
  import numpy as np
  from gensim.models import Word2Vec as wvec
  from sklearn.decomposition import PCA

  from asteval import Interpreter
  asteval2 = Interpreter()

  to_be_vectorized = []

  for rev in all_phrases1['cleaned_reviews']:
    to_be_vectorized.append(asteval2(rev))

  all_phrases1['cleaned_reviews2'] = to_be_vectorized

  cbow = wvec(min_count=10,window=5,size=all_phrases1.shape[0],alpha=0.01,min_alpha=0.001,sample=6e-5,negative = 10)


  dictionary = gensim.corpora.Dictionary(all_phrases1['cleaned_reviews'])

  model = wvec(min_count=10,window=5,size=all_phrases1.shape[0],alpha=0.01,min_alpha=0.001,sample=6e-5,negative = 10)

  model.build_vocab(sentences=all_phrases1['cleaned_reviews'])

  model.train(all_phrases1,total_examples=cbow.corpus_count,epochs=200,)

  vectors = model.wv.vectors

  print(vectors.shape)

  pca = PCA(n_components=100)

  v2 = vectors.T
  # v2 = vectors


  print(v2.shape)


  v2 = pca.fit_transform(v2) 

  return v2

# v2 = np.array(v2)




# tfidf_embedding

In [ ]:
def tfidf_embedding(phrases):
  from sklearn.feature_extraction.text import TfidfVectorizer
  import numpy as np
  from tensorflow.keras.preprocessing.sequence import pad_sequences
  tfidf = TfidfVectorizer(min_df=1)
  import asteval
  from asteval import Interpreter

  interpreter = Interpreter()
  
  list_reviews = []

  for rev in phrases['cleaned_reviews']:

    j = interpreter(rev)

    # print(type(j))

    list_reviews.append(j)

  # print(type(list_reviews))

  # tfidf.fit(phrases['cleaned_reviews'])

  tidy_reviews = []

  # for rev in phrases['cleaned_reviews']:
  for rev in list_reviews:

    # print(type(rev))
    
    # j = ' '.join(rev)
    j = ' '.join(rev)
    # print('j:',j)


    tidy_reviews.append(j)
    # print('tidy_reviews:',tidy_reviews)

  print('tidy reviews:',tidy_reviews[:5])

  # return tidy_reviews

  # tidy_reviews = phrases['cleaned_reviews'].tolist()
  print('vectorizing...')
  # vectors = tfidf.transform(phrases['cleaned_reviews'][0])
  vectors = tfidf.fit_transform(tidy_reviews)

  vectors = vectors.toarray()

  padded_vectors = pad_sequences(vectors,padding="post",truncating="post",maxlen=10000)
  # padded_vectors = pad_sequences(vectors,padding="post")

  # padded_vectors = pad_sequences(vectors,padding="post",truncating="post")




  # for i,review in enumerate(phrases['cleaned_reviews'][1:]):
  #   print(i+1)

  #   review = tfidf.transform(review)

  #   review = review.toarray()

  #   vectors = np.append(vectors,review,axis=0)

  print('vectors shape:',vectors.shape)
  print('padded_vectors shape:',vectors.shape)



  return padded_vectors

# Data splitting

In [ ]:
def split_data(vectors,labels):

  print('vectors:',vectors.shape)
  print('labels:',labels.shape)
  import numpy as np
  from keras.utils import to_categorical
  # df = phrases.sample(frac=1).reset_index(drop=True)

  # seen_data = vectors[:round(df.shape[0]*0.9),:]
  # unseen_data = vectors[round(df.shape[0]*0.9):,:]

  seen_vectors = vectors[:round(vectors.shape[0]*0.9)]
  seen_labels = labels[:round(vectors.shape[0]*0.9)]
  

  unseen_vectors = vectors[round(vectors.shape[0]*0.9):]
  unseen_labels = labels[round(vectors.shape[0]*0.9):]


  print('seen_vectors shape:',seen_vectors.shape)
  print('seen_labels shape:',seen_labels.shape)
  print('unseen_vectors shape:',unseen_vectors.shape)
  print('unseen_labels shape:',unseen_labels.shape)



  from sklearn.model_selection import train_test_split

  x_train,x_test,y_train,y_test = train_test_split(vectors,labels,test_size=0.1,random_state=0)
  print('data split')

  print('x_train shape',x_train.shape)
  print('y_train shape',y_train.shape)
  print('x_test shape',x_test.shape)
  print('y_test shape',y_test.shape)

 

  x_train = x_train.reshape((x_train.shape[0],x_train.shape[1],1))
  x_test = x_test.reshape((x_test.shape[0],x_test.shape[1],1))

  y_train = np.array(y_train)
  y_test = np.array(y_test)
  # y_train = y_train.reshape((-1,1))
  # y_test = y_test.reshape((-1,1))

  # y_train = to_categorical(y_train,3)
  # y_test = to_categorical(y_test,3)

  model = training(x_train,y_train,x_test,y_test)
  # training(x_train,y_train)
  return model






# Training

In [ ]:
def training(x_train,y_train,x_test,y_test): # Add unseen_vectors,unseen_labels

  print('In training')
  import keras
  from keras.layers import LSTM,Dropout,Flatten,Dense,Bidirectional
  from keras.models import Sequential
  import numpy as np
  from keras.callbacks import ReduceLROnPlateau
  from keras.utils import to_categorical
  from sklearn.metrics import accuracy_score,confusion_matrix

  print('x_train shape:',x_train.shape)
  print('y_train shape:',y_train.shape)
  print('x_test shape:',x_test.shape)
  print('y_test shape:',y_test.shape)


  weights = {1:10000,0:1000,2:100}

  lstm = Sequential()
  lstm.add(Bidirectional(LSTM(32,input_shape=(x_train.shape[1],x_train.shape[2]),activation='tanh',return_sequences=True)))
  lstm.add(Dropout(0.5))
  # lstm.add(Bidirectional(LSTM(32,activation='sigmoid',return_sequences=True)))
  # lstm.add(Dropout(0.2))
  # lstm.add(keras.layers.BatchNormalization())
  # lstm.add(Bidirectional(LSTM(32,activation='tanh',return_sequences=True)))
  # lstm.add(Dropout(0.2))
  # lstm.add(Bidirectional(LSTM(32,activation='tanh',return_sequences=True)))
  # lstm.add(Dropout(0.2))
  lstm.add(Flatten())

  lstm.add(Dense(3))
  lstm.add(keras.layers.Activation('softmax'))
  # lstm.add(Dense(1,activation='sigmoid'))


  # lstm.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
  lstm.compile(optimizer=keras.optimizers.Adam(lr=0.001),loss='categorical_crossentropy',metrics=['accuracy'])

  reduce_lr = ReduceLROnPlateau(monitor='loss',factor=0.01,patience=5,min_lr=0.0001)


  # print(lstm.summary())
  lstm.fit(x_train,y_train,epochs=5,callbacks=[reduce_lr],steps_per_epoch=250)

  print('evaluating:')

  # lstm.evaluate(x_test,y_test)

  model = predict(lstm,x_test,y_test)

  return lstm



 

In [ ]:
def predict(lstm,x_test,y_test):

  import keras
  from keras.utils import to_categorical

  # lstm = keras.models.load_models('/content/drive/MyDrive/NLP Project/imdb_classifier_2.0.h5')

  from sklearn.metrics import accuracy_score,confusion_matrix

  pred = lstm.predict(x_test)

  # lstm.save('imdb_classifier.h5')

  print('shape of pred:',pred.shape)


  # # pred = pred[0][:]
  print('0th index:',(pred[0]))
  print('index value:',np.where(np.max(pred[0][:])))

  revs = []


  for i in range(len(pred)):
    # if((pred[i][0])>pred[i][1]):
    #   if(pred[i][0]>pred[i][2]):
    #     revs.append(0)

    # elif(pred[i][1]>pred[i][2]):
    #   revs.append(1)

    # else:
    x = np.where(np.max(pred[i][:]))
    revs.append(x[0])

  #   else:
  #     revs.append(0)


  revs = np.array(revs)

  revs = to_categorical(revs,3)

  # revs = revs.reshape((-1,1))

  print('shape of revs:',revs.shape)


  print('Accuracy on testing data:',accuracy_score(revs,y_test))

  print('\n')

  print(confusion_matrix(y_test.argmax(axis=1),revs.argmax(axis=1)))

  return lstm


# Driver Function

In [ ]:
def get_movie():
  load_data()
  movie = input('Enter the name of the movie whose reviews you want to see:')
  movie = movie.lower()
  # movie = 'tom horn'
  # info = selection
  info = movie.title()
  info = info+' '
  print(info)

  if (info not in movie_links['title']):
    print('Movie not recognized')
    return 0,1
  id = str(movie_links.imdbId.loc[info])
  id = 'tt0'+id
  wd = webdriver.Chrome('chromedriver',options=options)

  wd.get('https://www.imdb.com/title/{}/reviews?ref_=tt_urv'.format(id))

  print('Extending the page...')
  
  page = extend_page(wd)

  reviews = scrape_reviews(page)

  if(len(reviews)>=200):
    print('Labelling the reviews...')
    phrases = labelling(reviews)
    print('preprocessing the reviews...')
    phrases['cleaned_reviews'] = preprocess(phrases['reviews'])

    print('Embedding...')

    # vectors = tfidf_embedding(phrases)
    vectors = embedding(phrases)


    print('Data collection and preprocessing complete.')

    # v2 = vectors[:100,:]
    # phrases = phrases.iloc[:100,:]
    # print('Shape of vectors',vectors.shape)
    print('Shape of phrases',phrases.shape)

    return phrases,vectors
    # return phrases


  else:
    print('There are not enough reviews given for this movie.')
    return 0,1
    # return 0


  


In [ ]:
phrases,vectors = get_movie()

Enter the name of the movie whose reviews you want to see:iron man
Iron Man 
Extending the page...
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1


336it [00:00, 69164.02it/s]
189it [00:00, 69366.77it/s]


525 reviews scraped
Labelling the reviews...
labelling
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


/usr/local/lib/python3.7/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


reviews: 525
labels: 525
preprocessing the reviews...
text has been cleaned
text has been lemmatized
bigram step 1
bigram step 2
bigrams have been created
type: <class 'gensim.interfaces.TransformedCorpus'>
Embedding...
embedding
Shape of phrases (525, 3)


/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
TypeError
   ['great', 'movie', 'serf', 'perfect', 'introduction', 'tony_stark', 's', 'character', 'antagonist', 'bit', 'boring', 'loved', 'movie']
Runtime Error
TypeError
   ['wow', 'wow', 'wow', 'movie', 'waiting', 'long_time', 'avid', 'ironman', 'reader', 'collected', 'comic', 'life', 'latest', 'year', 'ironman', 'kid', 'friend', 'ironman', 'movie', 'laughed', 'said', 'special_effect', 'look', 'ridiculous', 'mind', 'like', 'early', 's', 'age', 'cgi', 'age', 'ironman', 'fan', 'come', 'life', 'spectacular', 'graphic', 'reason', 'spend', 'hard', 'earned', 'plot', 'pretty', 'thought', 'acting', 'fine', 'better', 

(1182, 525)
(525, 1182)
Data collection and preprocessing complete.
Shape of phrases (525, 4)


In [ ]:
model = split_data(vectors,np.array(phrases['label']))

vectors: (525, 100)
labels: (525,)
seen_vectors shape: (472, 100)
seen_labels shape: (472,)
unseen_vectors shape: (53, 100)
unseen_labels shape: (53,)
data split
x_train shape (472, 100)
y_train shape (472,)
x_test shape (53, 100)
y_test shape (53,)
In training
x_train shape: (472, 100, 1)
y_train shape: (472,)
x_test shape: (53, 100, 1)
y_test shape: (53,)
Epoch 1/5


ValueError: ignored

In [ ]:
subset_phrases = phrases
subset_vectors = vectors

In [ ]:
import numpy as np
from sklearn.utils import resample
  # df = phrases.sample(frac=1).reset_index(drop=True)

seen_data = subset_phrases.iloc[:round(subset_phrases.shape[0]*0.9),:]
unseen_data = subset_phrases.iloc[round(subset_phrases.shape[0]*0.9):,:]

seen_vectors = subset_vectors[:round(subset_vectors.shape[0]*0.9),:]

unseen_vectors = subset_vectors[round(subset_vectors.shape[0]*0.9):,:]

from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test = train_test_split(seen_vectors,seen_data['label'],test_size=0.1,random_state=0)


x_train = x_train.reshape((x_train.shape[0],x_train.shape[1],1))
x_test = x_test.reshape((x_test.shape[0],x_test.shape[1],1))



y_train = np.array(y_train)
y_test = np.array(y_test)
y_train = y_train.reshape((-1,1))
y_test = y_test.reshape((-1,1))


In [ ]:
from keras.utils import to_categorical
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)
print('positive class in training set:',len(y_train[y_train==1]))
print('negative class in training set:',len(y_train[y_train==2]))
print('neutral class in training set:',len(y_train[y_train==0]))


print('positive class in testing set:',len(y_test[y_test==1]))
print('negative class in testing set:',len(y_test[y_test==2]))
print('neutral class in testing set:',len(y_test[y_test==0]))

pos_train = x_train[y_train[y_train==1]]
neg_train = x_train[y_train[y_train==2]]
neu_train = x_train[y_train[y_train==0]]


print(pos_train.shape)
print(neg_train.shape)
print(neu_train.shape)

upsampled_neg = resample(neg_train,n_samples=5000,replace=True)
upsampled_pos = resample(pos_train,n_samples=5000,replace=True)
upsampled_neu = resample(neu_train,n_samples=5000,replace=True)


print('upsampled_neg shape:',upsampled_neg.shape)
print('upsampled_pos shape:',upsampled_pos.shape)
print('upsampled_neu shape:',upsampled_neu.shape)

all_vectors2 = np.append(upsampled_neg,upsampled_pos,axis=0)
all_vectors2 = np.append(all_vectors2,upsampled_neu,axis=0)
print('all_vectors2 shape:',all_vectors2.shape)
pos_y_train = y_train[y_train==1]
neg_y_train = [2 for i in range(len(upsampled_neg))]
pos_y_train = [1 for i in range(len(upsampled_pos))]
neu_y_train = [0 for i in range(len(upsampled_neu))]
print('neg_y_train len',len(neg_y_train))
print('pos_y_train len',len(pos_y_train))
print('neu_y_train len',len(neu_y_train))


new_y_train = np.append(neg_y_train,pos_y_train)
new_y_train = np.append(new_y_train,neu_y_train)

new_y_train = to_categorical(new_y_train,3)

# print(new_y_train.shape)
# new_y_train = new_y_train.reshape((-1,1))
print('new_y_train shape',new_y_train.shape)

In [ ]:
from sklearn.utils import shuffle
all_vectors2,new_y_train = shuffle(all_vectors2,new_y_train,random_state=0)

In [ ]:
model = split_data(all_vectors2,new_y_train)

In [ ]:
model = predict(model,all_vectors2,new_y_train)

In [ ]:
  model.save('imdb_classifier_2.1.h5')

In [ ]:
labels = np.array(phrases['label'])

In [ ]:
print('vectors:',vectors.shape)
print('labels:',labels.shape)
import numpy as np
from keras.utils import to_categorical
# df = phrases.sample(frac=1).reset_index(drop=True)

# seen_data = vectors[:round(df.shape[0]*0.9),:]
# unseen_data = vectors[round(df.shape[0]*0.9):,:]

seen_vectors = vectors[:round(vectors.shape[0]*0.9)]
seen_labels = labels[:round(vectors.shape[0]*0.9)]


unseen_vectors = vectors[round(vectors.shape[0]*0.9):]
unseen_labels = labels[round(vectors.shape[0]*0.9):]


print('seen_vectors shape:',seen_vectors.shape)
print('seen_labels shape:',seen_labels.shape)
print('unseen_vectors shape:',unseen_vectors.shape)
print('unseen_labels shape:',unseen_labels.shape)



from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test = train_test_split(vectors,labels,test_size=0.1,random_state=0)
print('data split')

print('x_train shape',x_train.shape)
print('y_train shape',y_train.shape)
print('x_test shape',x_test.shape)
print('y_test shape',y_test.shape)



x_train = x_train.reshape((x_train.shape[0],x_train.shape[1],1))
x_test = x_test.reshape((x_test.shape[0],x_test.shape[1],1))

y_train = np.array(y_train)
y_test = np.array(y_test)
# y_train = y_train.reshape((-1,1))
# y_test = y_test.reshape((-1,1))

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# training(x_train,y_train,x_test,y_test)
# training(x_train,y_train)


In [ ]:
import keras
from keras.layers import LSTM,Dropout,Flatten,Dense,Bidirectional
from keras.models import Sequential
import numpy as np
from keras.callbacks import ReduceLROnPlateau
from keras.utils import to_categorical
from sklearn.metrics import accuracy_score,confusion_matrix

print('x_train shape:',x_train.shape)
print('y_train shape:',y_train.shape)
print('x_test shape:',x_test.shape)
print('y_test shape:',y_test.shape)


weights = {1:10000,0:1000,2:100}

lstm = Sequential()
lstm.add(Bidirectional(LSTM(32,input_shape=(x_train.shape[1],x_train.shape[2]),activation='tanh',return_sequences=True)))
lstm.add(Dropout(0.2))
# lstm.add(Bidirectional(LSTM(32,activation='tanh',return_sequences=True)))
# lstm.add(Dropout(0.2))

# lstm.add(Bidirectional(LSTM(32,activation='tanh',return_sequences=True)))
# lstm.add(Dropout(0.2))
lstm.add(Bidirectional(LSTM(32,activation='tanh',return_sequences=True)))
lstm.add(Dropout(0.2))
lstm.add(Flatten())

lstm.add(Dense(3,activation='softmax'))
# lstm.add(Dense(1,activation='sigmoid'))


# lstm.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
lstm.compile(optimizer=keras.optimizers.Adam(lr=0.001),loss='categorical_crossentropy',metrics=['accuracy'])

reduce_lr = ReduceLROnPlateau(monitor='loss',factor=0.01,patience=5,min_lr=0.0001)


# print(lstm.summary())
# lstm.fit(x_train,y_train,epochs=50,callbacks=[reduce_lr])

# print('evaluating:')

# # lstm.evaluate(x_test,y_test)

# pred = lstm.predict(x_test)

# # lstm.save('imdb_classifier.h5')

# print('shape of pred:',pred.shape)


# # # pred = pred[0][:]
# print('0th index:',(pred[0]))
# # print('mean value:',np.mean(pred[0][:]))

# revs = []


# for i in range(len(pred)):
#   if((pred[i][0])>pred[i][1]):
#     if(pred[i][0]>pred[i][2]):
#       revs.append(0)

#   elif(pred[i][1]>pred[i][2]):
#     revs.append(1)

#   else:
#     revs.append(2)

# #   else:
# #     revs.append(0)


# revs = np.array(revs)

# revs = to_categorical(revs,3)

# # revs = revs.reshape((-1,1))


# print('Accuracy on testing data:',accuracy_score(revs,y_test))

# print('\n')

# print(confusion_matrix(y_test.argmax(axis=1),revs.argmax(axis=1)))


In [ ]:
# l1 = list(phrases['label'])
# print(set(l1))

In [ ]:
# split_data(phrases,vectors)

# Data Collection

In [ ]:
load_data()

In [ ]:
def data_collection():
  import numpy as np
  import random
  import pandas as pd

  global all_phrases
  global all_vectors
  global already_selected
  global phrases
  global vectors


  already_selected = []
  final_selection = []
  all_phrases = 0

  # selection = 'John Wick '


  while (type(all_phrases)==int):

    selection = random.choice(movie_links['title'])
    # all_phrases,all_vectors = get_movie(selection)
    all_phrases = get_movie(selection)
    print(all_phrases)
    already_selected.append(selection)


  # print('returned from get_movie')

  # if (type(all_phrases)==int):
  #   # print('all_phrases is 0')
  #   all_phrases,all_vectors = data_collection()

  # else:

  final_selection.append(selection)
  print('Initial Shape of all_phrases:',all_phrases.shape)
  print('Initial Shape of all_vectors:',all_vectors.shape)

    # count = 1

  while(len(final_selection)!=100):

    all_phrases,already_selected,final_selection = data_collection_2(all_phrases,already_selected,final_selection)


  print('all_phrases2:',all_phrases.shape)
  print('already_selected:',len(already_selected))
  print('LENGTH OF FINAL_SELECTION:',len(final_selection))
  print('movies selected:',final_selection)

  return all_phrases


In [ ]:
def data_collection_2(all_phrases,already_selected,final_selection):

  import random
  import numpy as np
  import pandas as pd



  phrases = 0

  print('In data_collection_2:')




  while (type(phrases)==int):
    print('all_phrases is 0, continuing')
    selection = random.choice(movie_links['title'])
    if (selection not in already_selected):
      already_selected.append(selection)
      phrases = get_movie(selection)


    
  final_selection.append(selection)

  
  print('ELngth of final_selection:',len(final_selection))
  print('Shape of phrases:',phrases.shape)
  print('Shape of all_phrases:',all_phrases.shape)

  all_phrases = pd.concat([all_phrases,phrases],axis=0)




  return all_phrases,already_selected,final_selection



In [ ]:
all_phrases1 = data_collection()


In [ ]:
all_phrases1.to_csv('model_movies,csv')